## proc directory
/proc: directory in charge of linux performance optimization

proc fidrectory has 3 parts
1. PID directory
2. information files /meminfo: prevents information from kernel
3. /sys: tunables

## sys
important interface for sys
- fs: filesystem
- kernel: for kernel
- net: networking
- vm: memory

swappiness: the williness of the kernel to swap
out memory pages, if memory pressure arises

### Systemctl service
```systemctl status systemd-sysct```
its configuration is in /etc/sysctl.conf<br>
changes to system is applied on boot<br>
you can change swappiness

Useful sysctl command

In [ ]:
%%bash
sysctl --help
sysctl -p /etc/sysctl.conf

In [ ]:
%%bash
# list sysctl prameters
sysctl -a

# Managing Kernel Module Parameters
- Use **modinfo** to find which parameters are available
- Use **modprobe module key=value** to specify a parameter in runtime
- Use /etc/modprobe.d/modulename.conf to specify a permanent parameter

## Limiting Resource Usage

### ulimit
ulimit is the old way of configuring resource usage<br>
Applying POSIX Resource Limits
- Set runtime limits with **ulimit**
- - It dapplies to the shell in which the command is used
- Apply persistent ulimit settings to /etc/security/limits.conf
- Soft limits can be modified by the user, hard limits implements an absolute ceiling
- - Users can set soft limits, but only lower than the enforced hard limit

In [ ]:
%%bash
cat /etc/security/limits.conf

#### Managing Persistent limits
- **pam_limits** is applied from PAM sessions to configure persistent limits
- It works with /etc/security/limits.conf and /etc/security/limits.d/*.conf
- See *man 5 limits.conf** for documentation
- - Notice that not all llimites currently work on RHEL. The **rss**limit for instance doesn not work
- Example: set the maximum amount of logins for studentst to 2, creating a file with the contents /etc/security/limits/students.d and the following contents:
- - **@students hard maxlogins 2**

#### Setting Limits to Services
- In systemd units, add the **LIMIT\** entries to the **[Service]** block of a unit file to limit what services can do
- If, for inatance you want to allow your blah.service a maximum of 60 seconds of CPU time before it is killed, add the following to **/etc/systemd/system/blah.service.d/10-cpulimits.conf**

[Service]
LimitCPU=60

- Use **systemctl daemon-reload** and **systemctl restart blah** to make the changes effective
- See **man5 systemd.exec** and **man 2 setrlimit** for a full list of Limit\* settings

PAM (Pluggable Authentication Modules): it shapes what is happening when user log into the system

<img src='screenshots/PAM.png'>

In [ ]:
%%bash
man systemd.exec | grep limits

In [ ]:
! mkdir /etc/systemd/system/sleep.service.d

In [ ]:
%%bash
# to reload
systemctl daemon-reload
systemctl restart
status sleep